In [3]:
import numpy as np
import string

from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

from qiskit_addon_cutting import partition_problem
from qiskit_addon_cutting import generate_cutting_experiments

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2, Batch

In [49]:
# constructing quantum circuit of same structure as given in the qasm file

num_qubits = 10
reps = 4
gate_sequence = ['rz','sx','rz','sx','x','rz']

# nearest neighbour configuration
seq_pairs = [(i,(i+1)%num_qubits) for i in range(0,num_qubits,2)]
warp_pairs = [(i,(i-1)%num_qubits) for i in range(0,num_qubits,2)]

# qc two local
qc = TwoLocal(num_qubits,rotation_blocks=['rz','sx','rz','sx','x','rz'],entanglement_blocks='cz', entanglement=[seq_pairs,warp_pairs], reps=2,skip_final_rotation_layer=True).decompose()  
qc.assign_parameters([np.random.random(1)[0] for i in range(len(qc.parameters))], inplace=True)
#qc.measure_all()
qc.draw(scale=0.8)

┌─────────────┐┌────┐┌─────────────┐ ┌────┐┌───┐┌─────────────┐    »
q_0: ┤ Rz(0.90927) ├┤ √X ├┤ Rz(0.20964) ├─┤ √X ├┤ X ├┤ Rz(0.12057) ├──■─»
     ├─────────────┤├────┤└┬────────────┤ ├────┤├───┤├─────────────┤  │ »
q_1: ┤ Rz(0.40053) ├┤ √X ├─┤ Rz(0.8629) ├─┤ √X ├┤ X ├┤ Rz(0.68287) ├──■─»
     ├─────────────┤├────┤ ├────────────┤ ├────┤├───┤└┬────────────┤    »
q_2: ┤ Rz(0.98815) ├┤ √X ├─┤ Rz(0.5019) ├─┤ √X ├┤ X ├─┤ Rz(0.6533) ├──■─»
     ├─────────────┤├────┤┌┴────────────┤ ├────┤├───┤┌┴────────────┤  │ »
q_3: ┤ Rz(0.59944) ├┤ √X ├┤ Rz(0.77262) ├─┤ √X ├┤ X ├┤ Rz(0.49437) ├──■─»
     ├─────────────┤├────┤├─────────────┤ ├────┤├───┤├─────────────┴┐   »
q_4: ┤ Rz(0.52737) ├┤ √X ├┤ Rz(0.95676) ├─┤ √X ├┤ X ├┤ Rz(0.062038) ├─■─»
     ├─────────────┤├────┤├─────────────┤ ├────┤├───┤├─────────────┬┘ │ »
q_5: ┤ Rz(0.72872) ├┤ √X ├┤ Rz(0.55804) ├─┤ √X ├┤ X ├┤ Rz(0.57077) ├──■─»
     ├─────────────┤├────┤├─────────────┤ ├────┤├───┤├─────────────┤    »
q_6: ┤ Rz(0.33631) ├┤ √X ├┤ Rz(0.92156) ├─┤ √X ├┤ X ├┤ Rz(0.23558) ├──■─»
     ├─────────────┤├────┤├─────────────┴┐├────┤├───┤├─────────────┤  │ »
q_7: ┤ Rz(0.57289) ├┤ √X ├┤ Rz(0.020444) ├┤ √X ├┤ X ├┤ Rz(0.93208) ├──■─»
     └┬────────────┤├────┤├─────────────┬┘├────┤├───┤└┬────────────┤    »
q_8: ─┤ Rz(0.9284) ├┤ √X ├┤ Rz(0.69014) ├─┤ √X ├┤ X ├─┤ Rz(0.7849) ├──■─»
     ┌┴────────────┤├────┤├─────────────┤ ├────┤├───┤┌┴────────────┤  │ »
q_9: ┤ Rz(0.35969) ├┤ √X ├┤ Rz(0.28182) ├─┤ √X ├┤ X ├┤ Rz(0.27003) ├──■─»
     └─────────────┘└────┘└─────────────┘ └────┘└───┘└─────────────┘    »
«     ┌─────────────┐┌────┐ ┌────────────┐┌────┐┌───┐┌─────────────┐       
«q_0: ┤ Rz(0.36739) ├┤ √X ├─┤ Rz(0.2869) ├┤ √X ├┤ X ├┤ Rz(0.56366) ├─────■─
«     ├─────────────┤├────┤┌┴────────────┤├────┤├───┤├─────────────┤     │ 
«q_1: ┤ Rz(0.83165) ├┤ √X ├┤ Rz(0.48539) ├┤ √X ├┤ X ├┤ Rz(0.27787) ├──■──┼─
«     ├─────────────┤├────┤├─────────────┤├────┤├───┤├─────────────┤  │  │ 
«q_2: ┤ Rz(0.76633) ├┤ √X ├┤ Rz(0.56164) ├┤ √X ├┤ X ├┤ Rz(0.73417) ├──■──┼─
«     ├─────────────┤├────┤├─────────────┤├────┤├───┤├─────────────┤     │ 
«q_3: ┤ Rz(0.47225) ├┤ √X ├┤ Rz(0.59136) ├┤ √X ├┤ X ├┤ Rz(0.33197) ├──■──┼─
«     ├─────────────┤├────┤└┬────────────┤├────┤├───┤├─────────────┴┐ │  │ 
«q_4: ┤ Rz(0.43518) ├┤ √X ├─┤ Rz(0.2933) ├┤ √X ├┤ X ├┤ Rz(0.092629) ├─■──┼─
«     ├─────────────┤├────┤┌┴────────────┤├────┤├───┤├─────────────┬┘    │ 
«q_5: ┤ Rz(0.22111) ├┤ √X ├┤ Rz(0.20169) ├┤ √X ├┤ X ├┤ Rz(0.82372) ├──■──┼─
«     ├─────────────┤├────┤├─────────────┤├────┤├───┤├─────────────┤  │  │ 
«q_6: ┤ Rz(0.79831) ├┤ √X ├┤ Rz(0.44107) ├┤ √X ├┤ X ├┤ Rz(0.73655) ├──■──┼─
«     ├─────────────┤├────┤└┬────────────┤├────┤├───┤├─────────────┤     │ 
«q_7: ┤ Rz(0.39977) ├┤ √X ├─┤ Rz(0.9843) ├┤ √X ├┤ X ├┤ Rz(0.16198) ├──■──┼─
«     ├─────────────┤├────┤┌┴────────────┤├────┤├───┤├─────────────┤  │  │ 
«q_8: ┤ Rz(0.55267) ├┤ √X ├┤ Rz(0.39452) ├┤ √X ├┤ X ├┤ Rz(0.82759) ├──■──┼─
«     ├─────────────┤├────┤├─────────────┤├────┤├───┤├─────────────┤     │ 
«q_9: ┤ Rz(0.44736) ├┤ √X ├┤ Rz(0.96744) ├┤ √X ├┤ X ├┤ Rz(0.73508) ├─────■─
«     └─────────────┘└────┘└─────────────┘└────┘└───┘└─────────────┘

In [50]:
num_qubits = qc.num_qubits
num_qubits

10

In [51]:
def cutting_circuits(qc,num_cuts):

    num_qubits = qc.num_qubits
    observable = SparsePauliOp(["Z"*num_qubits])

    labels  = list(string.ascii_uppercase[:num_cuts])
    partition_string = ''.join([label*(num_qubits//num_cuts) for label in labels])

    partitioned_problem = partition_problem(
    circuit=qc, partition_labels=partition_string,observables=observable.paulis)

    subcircuits = partitioned_problem.subcircuits
    subobservables = partitioned_problem.subobservables
    bases = partitioned_problem.bases

    return(subcircuits,subobservables,bases)

In [52]:
subcircuits,subobservables,bases = cutting_circuits(qc,5)

In [53]:
num_samples = 10

subexperiments, coefficients = generate_cutting_experiments(
    circuits=subcircuits, observables=subobservables, num_samples=num_samples
)

In [54]:
backend = AerSimulator()

pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_subexperiments = {
    label: pass_manager.run(partition_subexpts)
    for label, partition_subexpts in subexperiments.items()
}

with Batch(backend=backend) as batch:
    sampler = SamplerV2(mode=batch)
    jobs = {
        label: sampler.run(subsystem_subexpts, shots=2**12).result()
        for label, subsystem_subexpts in isa_subexperiments.items()
    }

In [55]:
def knitting_results(job_dict): 

    final_state = ''

    for results in job_dict.values():
        trials = 0
        final_counts = {}
        for result in results:
            counts = result.data.observable_measurements.get_counts()
            trials += sum(list(counts.values()))
            for keys in counts.keys():
                final_counts[keys] = final_counts.get(keys, 0)+counts[keys]
        
        final_state+=max(final_counts,key=final_counts.get)
    
    return(final_state)

In [56]:
knitting_results(jobs)

'0000000000'

In [57]:
## Testing the output of original circuit
qc.measure_all()
sampler = SamplerV2(mode=backend)
result = sampler.run([qc],shots=2**12).result()

In [58]:
counts = result[0].data.meas.get_counts()
max(counts,key=counts.get)

'0000000000'

In [59]:
counts

{'0101000000': 42,
 '1011000000': 16,
 '0010001000': 39,
 '0000000000': 334,
 '0000010010': 41,
 '1101000000': 14,
 '0001000100': 23,
 '0100000000': 72,
 '0100001000': 30,
 '1000001100': 13,
 '0001001001': 6,
 '0000011000': 59,
 '0000001000': 138,
 '0010011010': 5,
 '1000000010': 41,
 '0000010000': 142,
 '0000000010': 93,
 '1000001010': 22,
 '0000100000': 49,
 '0101100010': 2,
 '0000000100': 53,
 '0011011000': 11,
 '0001010000': 77,
 '0001100000': 21,
 '1001000000': 54,
 '0001000010': 55,
 '1000011000': 35,
 '0001000000': 171,
 '1011010000': 11,
 '0100010001': 5,
 '0001001000': 75,
 '0010000010': 29,
 '1010010100': 3,
 '0001011000': 28,
 '0010001010': 11,
 '0000010110': 4,
 '0011000000': 42,
 '1110100100': 1,
 '0001001010': 23,
 '0100011000': 18,
 '1000000000': 142,
 '0001010010': 31,
 '0101011100': 1,
 '1000000100': 25,
 '0100011010': 5,
 '1000010000': 59,
 '0000000011': 2,
 '1010011000': 12,
 '1000010110': 5,
 '1000001000': 64,
 '0000011100': 9,
 '0100101000': 5,
 '1000101010': 2,
 '